# Transform_amendments

A partir da função _transform_legislations o texto será retornado com os tokens de emendas criados.

Essa função padroniza as emendas encontrados no texto, de forma que:

- emenda constitucional número 156 seja substituida por ST_EMENDA_156
- emendas regimentais nº.111 seja substituida por ST_EMENDA_111

In [1]:
import re

ST_PREFIX = "ST_"

def _transform_legislations(
    text,
    prefix_result,
    re_legislation,
    is_multiple=False,
    identification=None,
    previous_token=None,
    separator=None,
):
    "Create custom tokens for legislation"

    def build_token(m):
        if m.group("ignored_expression"):
            result = m.group("ignored_expression")
        else:
            result = ""
        result = (
            result
            + ST_PREFIX
            + prefix_result
            + m.group("identification")
            .replace('"', "")
            .replace("“", "")
            .replace("”", "")
            .replace(".", "")
            .replace(",", "")
            .replace("º", "")
            .lstrip("0")
            .upper()
        )
        return result

    def build_token_multi(m):
        result = (
            m.group("ignored_expression")
            + ST_PREFIX
            + prefix_result
            + m.group("identification")
            .replace('"', "")
            .replace("“", "")
            .replace("”", "")
            .replace(".", "")
            .replace(",", "")
            .replace("º", "")
            .lstrip("0")
            .upper()
        )
        return result

    # replaces first occurrences
    text = re_legislation.sub(build_token, text)

    # replaces multiple occurrences, if exists
    if is_multiple:
        regex_legislation_multi = re.compile(
            fr"(?:(?P<ignored_expression>{previous_token}{separator}){identification})",
            flags=re.I,
        )
        while regex_legislation_multi.findall(text) != []:
            text = regex_legislation_multi.sub(build_token_multi, text)

    return text


In [2]:
def transform_amendments(text, **kwargs):
    #Create custom tokens for amendments appearances in a text

    amendment_types = ("constituciona(?:l|is)", "regimenta(?:l|is)")
    amendment_types = "|".join(amendment_types)

    word_number = r"(?:n[oº]?\.?|n\.?º|n[úu]mero)"
    number = r"(?:\d+(?:[\.,]\d+)*)"
    year = fr"(?:[/-]{number})?"
    prefix_result = "EMENDA_"
    previous_token = fr"{ST_PREFIX}{prefix_result}{number}"
    separator = r"\s*[,e]\s+"
    amendment = fr"(?P<identification>{number}){year}"
    re_legislation = re.compile(
        fr"(?P<ignored_expression>)(emendas?(\s+({amendment_types}))?)\s+(?:{word_number}\s*)?{amendment}",
        flags=re.I,
    )

    return _transform_legislations(
        text,
        prefix_result,
        re_legislation,
        True,
        amendment,
        previous_token,
        separator,
    )


In [3]:
entradas = """
  Emenda Constitucional nº 45/2004
  Emendas Regimentais nº 3/2018 e 7/2019
"""

print("Entrada: ", entradas)

Entrada:  
  Emenda Constitucional nº 45/2004
  Emendas Regimentais nº 3/2018 e 7/2019



In [4]:
print("Saida: ", transform_amendments(entradas))

Saida:  
  ST_EMENDA_45
  ST_EMENDA_3 e ST_EMENDA_7



# DATASET VICTOR

In [5]:
import pandas as pd

test = pd.read_csv('./data/train_small.csv')
text = test['body']

In [7]:
print("Dados de entrada:\n",text)

Dados de entrada:
 0         {"tribunal justiça estado bahia poder judiciár...
1         {"excelentíssimo senhor doutor juiz direito ju...
2         {"razões recurso inominado recorrente atlantic...
3         {"empresa recorrente tornou credora dos débito...
4         {"entretanto verdade parte apelante tornou tit...
                                ...                        
149212    {"supremo tribunal federal recurso extraordiná...
149213    {"seção recursos extraordinários mandado intim...
149214    {"ttar qsvòwi a edewrr seção recursos extraord...
149215    {"ertidao certifico dou que dirigi setor autar...
149216    {"supremo tribunal federal secretaria judiciár...
Name: body, Length: 149217, dtype: object


In [8]:
lista = []
for item in text:
    lista.append(transform_amendments(item))

In [10]:
print("Dados de saida: \n", lista[31])

Dados de saida: 
 {"sendo assim não conformado com r sentença fls recorrida interpôs recurso inominado qual foi conhecido seu provimento negado pela egrégia turma recursal extensão estado bahia não restando alternativas recorrente senão interposição presente recurso extraordinário repercussão geral LEI_11418 patente existência repercussão geral questão ora ventilada ensejando interposição recurso extraordinário isso porque diariamente são ajuizadas incontáveis ações danos morais quais atualmente abarrotam poder judiciário que vê sucumbir lentidão causada pelo excesso demandas infundadas desnecessárias surpreendentemente despeito clareza texto legal evidenciando que recorrente nada poderia fazer mas cumprir contrato assumido com empresa cessionária com cada vez mais freqüência eles vêm sendo condenados ressarcir por supostos danos violando disposições inciso do ARTIGO_5º constituição federal ensejando manifestação definitiva deste colendo pretório excelso sobre questão pacificando admis